In [3]:
#!pip install pyarrow
#!pip install tensorflow

     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ---------------------------------------- 57.5/57.5 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/65.5 kB ? eta -:--:--
     ---------------------------------------- 65.5/65.5 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ----------- ---------------------------- 0.4/1.5 MB 13.2 MB/s eta 0:00:01
     ---------------------------- ----------- 1.1/1.5 MB 13.4 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 11.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/181.3 kB ? eta -:--:--
     ------------------------------------- 181.3/181.3 kB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
     ----------------------------------- -- 143.4/151.7 kB 8.3 MB/s eta 0:00:01
     -------------------------------------- 151.7/151.7 kB 3.0 MB/s eta 0:00:00
  

   ---------------------------------------- 0.0/442.0 kB ? eta -:--:--
   --------------------------------------  440.3/442.0 kB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 442.0/442.0 kB 6.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/183.3 kB ? eta -:--:--
   ---------------------------------------- 183.3/183.3 kB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/102.2 kB ? eta -:--:--
   ------------------------------------ --- 92.2/102.2 kB ? eta -:--:--
   ---------------------------------------- 102.2/102.2 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/226.7 kB ? eta -:--:--
   ---------------------------------------- 226.7/226.7 kB 7.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/84.9 kB ? eta -:--:--
   ---------------------------------------- 84.9/84.9 kB 1.6 MB/s eta 0:00:00


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,TensorDataset
import pandas as pd
import pyarrow
from torchvision import models, transforms
from sklearn.model_selection import train_test_split

C:\Users\41794\anaconda3\envs\ada\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] La procédure spécifiée est introuvable'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from scikeras.wrappers import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [3]:
#CONSTANTS
exercises_dict = {
    'Abduction': 0,
    'Bird': 1,
    'Bridge': 2,
    'Knee': 3,
    'Shoulder': 4,
    'Squat' : 5,
    'Stretch' : 6
}

In [4]:
dataset_df = pd.read_parquet('dataset.parquet', engine='pyarrow')

In [5]:
display(dataset_df)

,Participant,Exercise,Set,Camera,time(s),left_ankle_x,left_ankle_y,left_ankle_z,left_ear_x,left_ear_y,...,right_pinky_z,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_thumb_x,right_thumb_y,right_thumb_z,right_wrist_x,right_wrist_y,right_wrist_z
0,P04,Abduction,Correct,Frontal_Top,0.000000,0.009328,0.779305,0.187880,0.020658,-0.693419,...,-0.633111,-0.054347,-0.511760,-0.185118,-0.068159,-0.602015,-0.598093,-0.066565,-0.600690,-0.592768
1,P04,Abduction,Correct,Frontal_Top,0.033333,0.009353,0.779298,0.185050,0.021187,-0.693361,...,-0.630835,-0.054293,-0.511703,-0.184861,-0.068130,-0.602200,-0.596219,-0.066508,-0.600835,-0.590916
2,P04,Abduction,Correct,Frontal_Top,0.066667,0.009722,0.779311,0.179212,0.021351,-0.693388,...,-0.630923,-0.054289,-0.511696,-0.184849,-0.067241,-0.601852,-0.596222,-0.065676,-0.600436,-0.590910
3,P04,Abduction,Correct,Frontal_Top,0.100000,0.011696,0.778027,0.166566,0.023431,-0.693102,...,-0.631444,-0.053769,-0.511612,-0.183764,-0.062897,-0.600404,-0.596646,-0.061278,-0.599290,-0.591304
4,P04,Abduction,Correct,Frontal_Top,0.133333,0.011671,0.778045,0.166878,0.023658,-0.693346,...,-0.631149,-0.053763,-0.512331,-0.183866,-0.061787,-0.600463,-0.596128,-0.060647,-0.599425,-0.590991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,P28,Stretch,E,Side_Low,24.333333,0.104921,0.764555,0.082647,0.058097,-0.669222,...,-0.153244,-0.163026,-0.512648,-0.044837,-0.626770,-0.533090,-0.155961,-0.617913,-0.532957,-0.135954
731,P28,Stretch,E,Side_Low,24.366667,0.104855,0.764961,0.084220,0.058165,-0.668928,...,-0.157576,-0.163073,-0.512568,-0.046608,-0.626767,-0.531939,-0.159559,-0.617969,-0.531721,-0.139373
732,P28,Stretch,E,Side_Low,24.400000,0.105392,0.765160,0.084043,0.058210,-0.668465,...,-0.158064,-0.163069,-0.512303,-0.046589,-0.626679,-0.530991,-0.159817,-0.617878,-0.530680,-0.139622
733,P28,Stretch,E,Side_Low,24.433333,0.106408,0.765264,0.084009,0.059145,-0.668389,...,-0.158204,-0.162886,-0.512327,-0.046626,-0.625844,-0.529880,-0.159893,-0.617153,-0.529503,-0.139712


In [11]:
rows_with_nan = dataset_df[dataset_df.isna().any(axis=1)]

In [13]:
display(rows_with_nan)

,Participant,Exercise,Set,Camera,time(s),left_ankle_x,left_ankle_y,left_ankle_z,left_ear_x,left_ear_y,...,right_pinky_z,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_thumb_x,right_thumb_y,right_thumb_z,right_wrist_x,right_wrist_y,right_wrist_z
35,P04,Abduction,Correct,Frontal_Low,1.166667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572,P04,Abduction,Correct,Side_Low,19.066667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
573,P04,Abduction,Correct,Side_Low,19.100000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
602,P04,Abduction,Correct,Side_Low,20.066667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
739,P04,Abduction,Correct,Side_Low,24.633333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1510,P28,Bridge,B,Side_Low,50.333333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1511,P28,Bridge,B,Side_Low,50.366667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1514,P28,Bridge,B,Side_Low,50.466667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1515,P28,Bridge,B,Side_Low,50.500000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
interpolated_dataset_df = dataset_df.interpolate().copy()

C:\Users\41794\AppData\Local\Temp\ipykernel_13560\2081421524.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  interpolated_dataset_df = dataset_df.interpolate().copy()


In [17]:
rows_with_nan = interpolated_dataset_df[interpolated_dataset_df.isna().any(axis=1)]
display(rows_with_nan)

,Participant,Exercise,Set,Camera,time(s),left_ankle_x,left_ankle_y,left_ankle_z,left_ear_x,left_ear_y,...,right_pinky_z,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_thumb_x,right_thumb_y,right_thumb_z,right_wrist_x,right_wrist_y,right_wrist_z


In [6]:
def torch_helper(df, cols):
    """
    Obtain a NumPy array from a DataFrame.

    Parameters:
        df (pd.DataFrame): The input DataFrame with N rows.
        cols (list): The list of M columns to be included in the resulting NumPy array.

    Returns:
        torch tensor: A Tensor of dimension (NxM) containing the float values of the specified columns
    """
    return torch.tensor(df[cols].values, dtype=torch.float32)

In [7]:
# Get columns of X
columns = interpolated_dataset_df.columns.drop(['Participant','Exercise','Set','Camera'])
# Get X
X = torch_helper(interpolated_dataset_df,columns)

# Get labels
y = np.array([exercises_dict[e] for e in interpolated_dataset_df['Exercise']])

In [12]:
# Define the neural network model
def create_model():
    model = Sequential()
    model.add(Dense(64, input_dim=100, activation='relu'))
    for _ in range(5):
        model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [13]:
# Wrap the Keras model in a scikit-learn compatible classifier
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

# Define K-fold cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and evaluate the model
results = cross_val_score(model, X, y, cv=kfold)

# Print the cross-validation results
print("Cross-validation results: %.2f%% accuracy" % (results.mean() * 100))

C:\Users\41794\anaconda3\envs\ada\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\41794\anaconda3\envs\ada\Lib\site-packages\keras\src\layers\core\dense.py:73: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\41794\anaconda3\envs\ada\Lib\site-packages\keras\src\losses\losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)
C:\Users\41794\anaconda3\envs\ada\Lib\site-packa